# Adaptive partitioning algorithm for mutual information estimation

Python implementation of mutual information estimation, where adaptive partitioning strategy is applied.

## Reference
- Darbellay, G. A., & Vajda, I. (1999). Estimation of the information by an adaptive partitioning of the observation space. IEEE Transactions on Information Theory, 45(4), 1315-1321.

In [1]:
import numpy as np
from minfo import mutual_info as mi_cy
from minfo import tdmi as TDMI_cy
from minfo import tdmi_omp as TDMI_cy_omp
from mutual_info import mutual_info as mi_py
import time

def TDMI_py(dat, n):
    """Time-delay mutual information estimator. (Pure Python Version)
    
    Parameters:
        dat (np.ndarray) : 2D array of time series with 2 column.
            each column is a variable, and each row is a sample.
        n (int) : number of delays, including zero time-lag case.

    Returns:
        np.ndarray : 1d array of delayed mutual information series.

    """
    tdmi = np.zeros(n)
    N = dat.shape[0]
    for i in range(n):
        dat_buffer = np.zeros((N-i, 2))
        dat_buffer[:,0] = dat[:N-i,0]
        dat_buffer[:,1] = dat[i:,1]
        tdmi[i] = mi_py(dat_buffer)
    return tdmi

In [2]:
# load sample time series
dat = np.load('sample.npy', allow_pickle=True)
dat.shape

(10000, 2)

In [3]:
print('[INFO]: Testing mi (Python) ...')
%timeit mi_py(dat)

[INFO]: Testing mi (Python) ...
179 ms ± 5.65 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
print('[INFO]: Testing mi (Cython) ...')
%timeit mi_cy(dat[:,0], dat[:,1])

[INFO]: Testing mi (Cython) ...
11.1 ms ± 64 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
n_delay = 100
print('[INFO]: Testing tdmi (Python) ...')
%timeit TDMI_py(dat, n_delay)

[INFO]: Testing tdmi (Python) ...
402 ms ± 1.86 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
print('[INFO]: Testing tdmi (Cython) ...')
%timeit TDMI_cy(dat[:,0], dat[:,1], n_delay)

[INFO]: Testing tdmi (Cython) ...
167 ms ± 814 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
print('[INFO]: Testing tdmi (Cython/OpenMP) ...')
%timeit TDMI_cy_omp(dat[:,0], dat[:,1], n_delay)

[INFO]: Testing tdmi (Cython) ...
44 ms ± 399 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
